In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime, date
import os
import errno
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf
from pathlib import Path

simpler_sf.simple_salesforce()

In [2]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
# regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
hoy = date.today()
mes = hoy.month
ano = hoy.year


mesesDic = {
    1:'Enero',
    2:'Febrero',
    3:'Marzo',
    4:'Abril',
    5:'Mayo',
    6:'Junio',
    7:'Julio',
    8:'Agosto',
    9:'Septiembre',
    10:'Octubre',
    11:'Noviembre',
    12:'Diciembre'
}

mesesDic[mes]


def crear_carpeta(nombre_carpeta):
    if not os.path.isdir(str(nombre_carpeta) ):
        print("No existe, se crea")
        derty = os.mkdir(str(nombre_carpeta) )
        #nombre = str(nombre_carpeta) + str(mesesDic[mes]) + "/"
        print(derty)
        return derty
    else:
        print("Existe")
        nombre = str(nombre_carpeta) 
        return nombre

In [5]:

#campaña = "EY"
nombre_carpeta = "E:/Siempre/04-Comercial/09-FACTURACION/Datos_Para_Facturar/"
archivoDireccion = str(crear_carpeta(nombre_carpeta))

archivcarpeta_datos = "E:/Siempre/04-Comercial/09-FACTURACION/Datos_Para_Facturar/"
archivoDatos = str(crear_carpeta(archivcarpeta_datos))

print(archivoDireccion)

Existe
Existe
E:/Siempre/04-Comercial/09-FACTURACION/Datos_Para_Facturar/


In [6]:
def traerCampañas(campaña):    
    query_empresacontactoconvenio = "select Id,Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Empresa__r.Compania__c, Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña +"'"
    empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
    empresacontactoconvenio = empresacontactoconvenio.rename(
        columns={
            "Id": "Interno",
            "Paciente__r.IdentificationId__pc": "Rut Beneficiario",
            "Paciente__r.FirstName": "Nombres",
            "Paciente__r.LastName": "Apellidos",
            "N_Documento_Titular__c":"Rut Titular",
            "Empresa__r.P_liza__c":"Poliza",
            "Empresa__r.Identificador__c":"Empresa",
            "Convenio_Activo__c":"Activo",
            "Relaci_n_Con_Asegurado__c":"Relacion",
            "Empresa__r.Rut_Empresa__c":"Rut Empresa",
            "Empresa__r.Compania__c":"Empleador",
            
        }
    )
    
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].astype(str)
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.replace("-", "")
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.replace("–", "")
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].astype(str)
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.replace(".", "")
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.replace(" ", "")
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.upper()
    empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.strip()
    
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].astype(str)
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.replace("-", "")
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.replace("–", "")
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].astype(str)
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.replace(".", "")
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.replace(" ", "")
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.upper()
    empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.strip()
    
    derty = "F:/Empresas SF/reportes/" + campaña + "con_duplicados.xlsx"
    
    
    
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "163820064")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "153298939")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "191710940")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "182470082")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16277865")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16277865K")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "HEGJ930928HMNRRL06")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "16007258K")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "86676540")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "202201466")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "105233469")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "130949185")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "255356798")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "34005945")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "185410072")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "34954659")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "166110890")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "197415029")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "254468134")].index, inplace=True)
    empresacontactoconvenio.drop(empresacontactoconvenio[(empresacontactoconvenio["Rut Beneficiario"] == "27591679K")].index, inplace=True)
        
    #empresacontactoconvenio.head()
    empresacontactoconvenio["Fecha"] = mesesDic[mes]

    empresacontactoconvenio["Campaña"] = campaña
    empresacontactoconvenio["LLave"] = empresacontactoconvenio["Rut Titular"] + empresacontactoconvenio["Rut Beneficiario"]
    
    #Borrar duplicados
    empresacontactoconvenio = empresacontactoconvenio.drop_duplicates(subset=["Rut Titular","Poliza","Rut Beneficiario"],  keep="last")
    
    VguardarArchivo = archivcarpeta_datos + campaña + " " + mesesDic[mes] + " " + str(ano) + ".xlsx"
    empresacontactoconvenio.to_excel(VguardarArchivo, index=False)
    
        
    derty = "F:/Empresas SF/reportes/" + str(mesesDic[mes]) + "/"  
    crear_carpeta(derty)
    empresacontactoconvenio.to_excel(derty + "Base Beneficiarios " + campaña + ".xlsx" , index=False)
    
    joan = archivcarpeta_datos + str(mesesDic[mes]) + "/" 
    crear_carpeta(joan)
    empresacontactoconvenio.to_excel(joan + "Base Beneficiarios " + campaña + " " + str(mesesDic[mes]) + " " + str(ano) +"JK.xlsx" , index=False)
    #empresacontactoconvenio.shape()
    return 1


In [7]:
listado_campañas = [
    #"MetLife",
    #"Construmart",
    #"Milan Fajanovic",
    #"Socofar",
    #"THB Seguros",
    #"Lipigas",
    #"Farmacias Ahumada",
    #"EY",
    #"Minera El Abra",
    #"VidaCel",
    #"Dermik",
    #"Instruvalve",
    #"Grupo Psg",
    #"Pedro de Valdivia",
    #"Pesco",
    #"Talo",
    #"BICE Vida",
    #"Nexo",
    #"Lockton",
    #"Kaufmann",
    #"Sole Orienta",
    #"Farmacias Cruz Verde",
    #"BCI Contingencia",
    #"BCI Seguros Colectivos",
    #"BCI Seguros Colaboradores",
    #"BCI Seguros Individuales",
    #"EY",
    #"Vida Security",
    #"Zurich",
    #"MetLife Vida Individual",
    #"MetLife Agencia",
    #"MetLife Rentas Vitalicias",
    #"MetLife Rentas Vitalicias Falabella",
    "Willis Tower Watson",
    #"MetLife Gerencia Rentas Vitalicias",
    #"MetLife Producto Joven",
    #"MetLife Gerencia Rentas Vitalicias",
    #"MetLife Gerencia DM Falabella",
    #"MetLife Gerencia DM Rentas Vitalicias",
    #"Dermik"
    
]

#listado_campañas = ["Aldo Conti","Ambar","BBDO","Care U","Clinica Virtual","DDB","Grupo C&H","OMG", "Porter Novelli", "Productos San Juan", "Riddell Sports","Sintec","Ticsa","Transcontainer"]
# Crear dataframe a partir de una lista
df_campañas = pd.DataFrame(listado_campañas, columns=["Campaña"])
df_campañas.head()

,Campaña
0,Willis Tower Watson


In [8]:
df_campañas["Campaña"].apply(lambda y: traerCampañas(y))

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:16<00:00, 16.08s/it]
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21116\2843750796.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Beneficiario"] = empresacontactoconvenio["Rut Beneficiario"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21116\2843750796.py:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Titular"] = empresacontactoconvenio["Rut Titular"].str.replace(".", "")


No existe, se crea
None
No existe, se crea
None


0    1
Name: Campaña, dtype: int64